#### Tagging the articles (in MongoDB) containing LOC and ORG tag.

In [1]:
from pymongo import MongoClient

In [2]:
mongo = MongoClient(host='10.237.26.154', port=27017)

In [3]:
mongo.list_database_names()

['local', 'media-db']

In [4]:
mediaDb = mongo.get_database('media-db')

In [5]:
cursor = mediaDb.articles.find({"entities.type":"City"})

In [6]:
def updateText(text, entype, offset, length):
    if entype == 'Person':
        otag = '<PER>'
        ctag = '</PER>'
    elif entype == 'Company' or entype == 'Organization':
        otag = '<ORG>'
        ctag = '</ORG>'
    elif entype == 'City':
        otag = '<LOC>'
        ctag = '</LOC>'
    elif entype == 'ProvinceOrState':
        otag = '<STA>'
        ctag = '</STA>'
    else :
        otag = '<MISC>'
        ctag = '</MISC>'
    p1 = text[:offset]
    p2 = otag + text[offset:offset+length] + ctag
    p3 = text[offset+length:]
    
    shift = len(p2) - length

    return (p1 + p2 + p3, shift)

In [7]:
def updateOffset(entities, old_offset, shift):
    for e in entities:
        for inst in e['instances']:
            if inst['offset'] > old_offset:
                inst['offset']+=shift
    return entities

In [ ]:
entitiesNotFound = 0
noOfArticles = 0
fileName = './articles_loc_and_org.txt'
fout = open(fileName, 'w')
itr = 0
for c in cursor:
    text = c['text']
    ctr = 0
    entities = c['entities']
    cityPresent = False
    compPresent = False
    num_of_city = 0
    city_offset = 0
    while ctr < len(entities):
        e = entities[ctr]
        type_ = e['type']
        if type_ == 'City':
            cityPresent = True
            num_of_city += 1
        elif type_ == 'Company' or type_ == 'Organization':
            compPresent = True
        name = e['name']
        for i in e['instances']:
            if e['type'] == 'City':
                city_offset = 0
            leng = i['length']
            offset = i['offset']
            text, shift = updateText(text, type_, offset, leng)
            entities = updateOffset(entities, offset, shift)
        ctr += 1
    if num_of_city == 1 and city_offset == 0: # not a valid article- first word is generally the location of bureau
        pass
    elif cityPresent and compPresent:
        print(text, file=fout, flush=True)
        noOfArticles += 1
    itr += 1
print('Articles processed-', itr)
print('Articles containing both company and the location tags', noOfArticles)

In [ ]:
itr

In [ ]:
noOfArticles